In [ ]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

In [ ]:
!pip install matplotlib

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
image_path = "/content/drive/MyDrive/images/getz.jpg"
image = Image.open(image_path)

In [ ]:
prompt = """Create an image for an advertisement for Hyndai GetZ. The slogan for this ad is "peace of mind". The target audience is mothers with young families, single people, and over-50s whose offspring have left home. A description - The Getz. A little piece of mind from Hyundai."""

In [ ]:
inputs = processor(text=prompt, images=image, return_tensors="pt", padding=True)



In [ ]:
import torch

In [ ]:
# Generate CLIP similarity score
with torch.no_grad():
  outputs = model(**inputs)
  logits = outputs.logits_per_image.squeeze()
  similarity_score = logits.item()

In [ ]:
print(f"CLIP similarity score between prompt and image: {similarity_score}")


CLIP similarity score between prompt and image: 36.55706024169922


In [ ]:
!pip install konIQ-pytorch


ERROR: Could not find a version that satisfies the requirement konIQ-pytorch (from versions: none)
ERROR: No matching distribution found for konIQ-pytorch


In [ ]:
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm

# Load the InceptionV3 model pre-trained on ImageNet
inception_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=True)

# Function to preprocess images
def preprocess_image(image):
  image = tf.image.resize(image, (299, 299))
  image = tf.keras.applications.inception_v3.preprocess_input(image)
  return image

# Function to compute Inception Score
def compute_inception_score(images, batch_size=32):
  # Preprocess images
  preprocessed_images = np.array([preprocess_image(image) for image in images])

  # Predict class probabilities for each image
  logits = []
  for i in tqdm(range(0, len(images), batch_size)):
    batch = preprocessed_images[i:i+batch_size]
    logits.extend(inception_model.predict(batch))
  logits = np.array(logits)

  # Compute marginal distribution (average over all images)
  p_yx = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)

  # Compute marginal distribution (average over all images)
  p_y = np.mean(p_yx, axis=0, keepdims=True)

  # Compute Inception Score
  kl_divergence = np.sum(p_yx * (np.log(p_yx) - np.log(p_y)), axis=1)
  inception_score = np.exp(np.mean(kl_divergence))
  return inception_score

# Example usage
# Load images from a directory
image_directory = "/content/images"
image_files = os.listdir(image_directory)
images = [tf.keras.preprocessing.image.load_img(os.path.join(image_directory, img_file)) for img_file in image_files]

# Compute Inception Score
inception_score = compute_inception_score(images)
print("Inception Score:", inception_score)


  0%|          | 0/1 [00:00<?, ?it/s]

1/1 [==============================] - 5s 5s/step


100%|██████████| 1/1 [00:04<00:00,  4.83s/it]

Inception Score: 1.0001585
